# Trans-Effects with t tests for 8p

Here we look for genes NOT in the event discussed that are affected by the arm-level event. We will find these effects by performing a series of t-tests comparing the proteomic values of the patients with the event against those without the event.

## Setup (Install necessary packages)

We will start by importing necessary packages and collecting all of the proteomics data we will need to run the tests. The cancer types analyzed should have been determined in 01_event_basic_info where we determine which types of cancer seem to have the event we are looking at.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cptac
from scipy import stats
import cnvutils
import cptac.utils

In [2]:
# These variables specify which chromosome and arm we're working on, and whether to do cis or trans
CHROMOSOME = '8'
ARM = 'p'
CIS_OR_TRANS = "trans"

In [3]:
if ARM == "p":
    EVENT_COLUMN = "loss_event"
    EXCLUDE_COLUMN = "gain_event"
    EVENT_START = 0
    EVENT_END = 30794385
    
elif ARM == "q":
    EVENT_COLUMN = "gain_event"
    EXCLUDE_COLUMN = "loss_event"
    EVENT_START = 80794385
    EVENT_END = 130794385

else:
    raise ValueError("Invalid value for ARM variable.")

## Load data

In [4]:
# Load in the cptac data for each cancer type that you want to analyze.

br = cptac.Brca()
# cc = cptac.Ccrcc()
co = cptac.Colon()
# en = cptac.Endometrial()
# gb = cptac.Gbm()
hn = cptac.Hnscc()
ls = cptac.Lscc()
lu = cptac.Luad()
ov = cptac.Ovarian()

cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (/home/caleb/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py, line 9)


In [5]:
# Now we need to get the proteomics tables for each type of cancer to analyze.
proteomics = {
    "brca": br.get_proteomics(tissue_type="tumor"),
    "colon": co.get_proteomics(tissue_type="tumor"),
    "hnscc": hn.get_proteomics(tissue_type="tumor"),
    "lscc": ls.get_proteomics(tissue_type="tumor"),
    "luad": lu.get_proteomics(tissue_type="tumor"),
    "ovarian": ov.get_proteomics(tissue_type="tumor")
}

## Select the proteins we're interested in

If we're looking at cis effects, we select proteins within the event. If we're looking at trans effects, we select proteins outside of the event.

Note that the cnvutils.get_event_genes function uses Ensembl gene IDs for the Database_ID column, while the proteomics dataframes that have a Database_ID column use RefSeq protein IDs. So, when we're selecting the genes we want, we ignore the Database_ID column if it is present, and just use gene names.

In [6]:
selected_genes = cnvutils.\
get_event_genes(
    chrm=CHROMOSOME,
    event_start=EVENT_START,
    event_end=EVENT_END,
    cis_or_trans=CIS_OR_TRANS
)["Name"].\
drop_duplicates(keep="first")

for cancer_type in proteomics.keys():
    df = proteomics[cancer_type].transpose()
    
    if df.index.nlevels == 1:
        df = df[df.index.isin(selected_genes)]
    else:
        df = df[df.index.isin(selected_genes, level="Name")]

    proteomics[cancer_type] = df

## Append Event Data

We now append the data from the event table that should have been created in a previous notebook.

In [7]:
has_event = dict()
for cancer_type in proteomics.keys():
    df = proteomics[cancer_type]
    df = df.transpose()
    event = pd.read_csv(f'{cancer_type}_has_event.tsv', sep='\t', index_col=0)
    if EXCLUDE_COLUMN:
        event.drop(EXCLUDE_COLUMN, axis=1, inplace=True)
    event.index.rename('Name')
    df = df.join(event)
    df = df.dropna(subset=[EVENT_COLUMN])
    has_event[cancer_type] = df[EVENT_COLUMN]
    proteomics[cancer_type] = df

/home/caleb/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/home/caleb/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/home/caleb/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/home/caleb/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


## Run T-Tests

In [8]:
results_df = None
for cancer_type in proteomics.keys():
    prot_df = proteomics[cancer_type]
    results = cptac.utils.wrap_ttest(
        df=prot_df, 
        label_column=EVENT_COLUMN,
        correction_method="fdr_bh",
        return_all=True
    )
    results.set_index('Comparison', inplace=True)
    if isinstance(results.index[0], tuple):
        results[['Name', f'{cancer_type}_Database_ID']] = pd.DataFrame(
            results.index.values.tolist(),
            index=results.index
        )
        results.set_index(['Name', f'{cancer_type}_Database_ID'], inplace=True)
    else:
        results.index.name='Name'
    results.rename(columns={'P_Value': f'{cancer_type}_pvalue'}, inplace=True)
    if results_df is None:
        results_df = results
    else:
        results_df = results_df.join(results)

## Append Difference Data

We want to collect one more piece of information about the data. We want to find the difference between the averages of the two groups. This will tell us if there is a positive or negative coorelation between the event and the proteomic data and can give us some indication of the strength of the coorelation. 

In [9]:
def get_diff(col, event):
    has_event = col[event]
    invert_list = [not x for x in event]
    no_event = col[invert_list]
    event_avg = has_event.mean()
    no_event_avg = no_event.mean()
    return event_avg - no_event_avg

In [10]:
diff_df = None
for cancer_type in proteomics.keys():
    df = proteomics[cancer_type]
    df = df.drop(EVENT_COLUMN, axis=1)
    results = df.apply(lambda x: get_diff(x, has_event[cancer_type]))
    df = pd.DataFrame(results)
    if isinstance(df.index[0], tuple):
        df[['Name', f'{cancer_type}_Database_ID']] = pd.DataFrame(df.index.values.tolist(), index=df.index)
        df.set_index(['Name', f'{cancer_type}_Database_ID'], inplace=True)
    else:
        df.index.name='Name'
    df.rename(columns={0: f'{cancer_type}_diff'}, inplace=True)
    if diff_df is None:
        diff_df = df
    else:
        diff_df = diff_df.join(df)

## Join the tables and save

We now join the difference table and the results table together. We also save the table to a tsv for use in future analyses. 

In [11]:
results_df = results_df.join(diff_df)

In [12]:
# This will save the resulting table in the same directory as this notebook.
# Modify if you would like to save to a different location.
results_df.to_csv(f"{CHROMOSOME}{ARM}_{CIS_OR_TRANS}effects_ttest.tsv", sep='\t')